In [1]:
import pandas as pd
import numpy as np
import pymssql
import pyodbc

In [2]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')
odbc_conn = pyodbc.connect('Driver={IBM i Access ODBC Driver}; '
       'SYSTEM = 192.168.0.161;'
        'Hostname=M3; '
        'Port=21; '
        'Protocol=TCPIP; '
        'Database=M3; '
        'UID=pud; '
        'PWD = Pud142;'
        ,autocommit=True)

In [3]:
#test connection
query='SELECT Filename, LockPath, LockDomain, LockDate, Username,FullName, Email from [ePDM_Elcam].[dbo].[Documents] INNER JOIN [ePDM_Elcam].[dbo].[Users] on [ePDM_Elcam].[dbo].[Documents].UserID = [ePDM_Elcam].[dbo].[Users].UserID WHERE LockDate IS NOT NULL AND CurrentStatusid IS NOT NULL AND Deleted=0 ORDER BY Username ASC'
query2='SELECT distinct(MMITNO) FROM M3FDBPRD.MITMAS m '
all_items=pd.read_sql(query2,odbc_conn)
files = pd.read_sql(query, mssql_conn)
files.sort_values(by='LockDate', ascending=False, inplace=True)
#find if item number '015495DOL      ' is in the list of all items





c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
def find_file(item):
    item=item.strip()
    query =f"select DocumentID,Filename from Documents d where Filename like '%{item}%'"
    result = pd.read_sql(query, mssql_conn)
    print(result)
    return result
res=find_file('SW Preliminary Design Review')
# res.to_csv('test.csv')

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   DocumentID                                           Filename
0      493982  PRS-0003-Genotropin Injector pen-SW Preliminar...
1      494388  PRS-0003-Genotropin Injector pen-SW Preliminar...
2      473329  PRS-0003-Genotropin Injector pen-SW Preliminar...
3      543296  PRS-0003-Genotropin Injector pen-SW Preliminar...
4      543582  PRS-0003-Genotropin Injector pen-SW Preliminar...
5      543589  PRS-0003-Genotropin Injector pen-SW Preliminar...
6      491531  PRS-0003-Genotropin Injector pen-SW Preliminar...


In [7]:

res ={}
for i in range(len(files)):
    row=files.iloc[i]
    if row['Username'] not in res:
        res[row['Username']] ={'email':row['Email'],'fullname':row['FullName'],'count':0}
    res[row['Username']]['count']+=1
#sort by count
res = sorted(res.items(), key=lambda x: x[1]['count'], reverse=True)
username=[]
fullname=[]
email=[]
count_files=[]
for k in res:
    # print(k)
    username.append(k[0])
    fullname.append(k[1]['fullname'])
    email.append(k[1]['email'])
    count_files.append(k[1]['count'])
df = pd.DataFrame({'username':username,'fullname':fullname,'email':email,'count_files':count_files})
df.to_csv('files.csv',index=False)



In [9]:
df2=pd.read_csv("kimaat.moshlam.csv")
#remove all the rows that have no email address
df2=df2[df2['EmailAddress'].notnull()]
#remove all the rows that the email address is not end with @elcam.co.il
df2=df2[df2['EmailAddress'].str.endswith('@elcam.co.il')]
#convert all the email address to lower case
df2['EmailAddress']=df2['EmailAddress'].str.lower()
#find the rows that the display name is Gal Einhorn
df2[df2['ManagerEmail']=='amotz@elcam.co.il']


,DisplayName,EmailAddress,ManagerEmail
707,Vanya Dobrovolsky,vanya.d@elcam.co.il,amotz@elcam.co.il
755,Idan Naaman,idan.n@elcam.co.il,amotz@elcam.co.il
922,Ilan Kachler,ilan.k@elcam.co.il,amotz@elcam.co.il
938,Oded Benedik,oded.b@elcam.co.il,amotz@elcam.co.il
1116,Ziv Cohen,ziv.c@elcam.co.il,amotz@elcam.co.il
1207,Ohad Gottesman,ohad.g@elcam.co.il,amotz@elcam.co.il


In [1]:


message = Message(From="me@example.com",
                  To="you@example.com",
                  charset="utf-8")
message.Subject = "An HTML Email"
message.Html = """This email uses <strong>HTML</strong>!"""
message.Body = """This is alternate text."""

sender = Mailer('smtp.example.com')
sender.send(message)



ModuleNotFoundError: No module named 'mailer'